# Basic GAN (Generative Adverserial Network)

---

## Built with Python and PyTorch

Created By: Xavier De Carvalho     
Created On: 14/08/2021 20:50PM     
Upated By: N/A     
Updated On: N/A     
Version: GAN0.0.01

### Requirements

---

**Hardware:**
1. GPU

**Packages:**
1. PyTorch
2. Matplotlib
3. TDQM

### Import Packages

---


In [ ]:
# Import Packages
import torch, pdb # PyTorch and Python Debugger
from torch.utils.data import DataLoader
from torch import nn
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from tqdm.auto import tqdm # Progress Bar
import matplotlib.pyplot as plt
%matplotlib inline

print("Packages imported!")

### Visualization Function

---


In [ ]:
# Display a grid with generative images vs real images
def showGrid(tensor, ch=1, size=(28,28), num=16):
  # Detach variable from gradient computation and pass it to the CPU and restructure it
  data = tensor.detach().cpu().view(-1, ch, *size)
  # Make a grid and change order of dimensions
  grid = make_grid(data[:num], nrow=4).permute(1,2,0) # Order of channels must be reordered to visualize with `plt`
  plt.imshow(grid)
  plt.show()

print("Visualization function created!")

### Setup Params and HyperParams

---


In [ ]:
epochs = 500 # Temporarily hardcoded
cur_step = 0 # Start current step at 0
info_step = 300 # Store every (n) steps we want to show about the current loss values, and visualize the images generated by the creator

# Accumulate generator loss and discriminator loss and calculate their mean
mean_gen_loss = 0
mean_disc_loss = 0

z_dim = 64 # Latent space dimensionality
lr = 0.00001 # Learn rate
loss_func = nn.BCEWithLogitsLoss() # Cross Entropy Loss Function

bs = 128 # Batch size
device = 'cuda' # Processing device

# Iterator to get training batches
dataLoader = DataLoader(
  MNIST('.', download=True, transform=transforms.ToTensor()),
  shuffle=True,
  batch_size=bs
)

print("Params and HyperParams set!")

### Generator Model

---


In [ ]:
# Generator Block
def genBlock(inp, out):
  return nn.Sequential(
      nn.Linear(inp, out),
      nn.BatchNorm1d(out), # Normalize values from previous layer to make training more stable
      nn.ReLU(inplace=True) # Set negatives to zero and only pass positive valyues to create non-linear transformation
  )

print('genBlock function created!')

# Generator
class Generator(nn.Module):
  def __init__(self, z_dim=64, i_dim=784, h_dim=128):
    super().__init__()
    self.gen = nn.Sequential(
        genBlock(z_dim, h_dim), # Input Size is 64 and Exit Size is 128
        genBlock(h_dim, h_dim*2), # Input Size is 128 and Exit Size is 256
        genBlock(h_dim*2, h_dim*4), # Input Size is 256 and Exit Size is 512
        genBlock(h_dim*4, h_dim*8), # Input Size is 512 and Exit Size is 1024
        nn.Linear(h_dim*8, i_dim), # Input Size 1024 and Exit Size is 784 (28x28) / Size of images in MNIST dataset
        nn.Sigmoid() # Set values to 0<>1
    )

  def forward(self, noise):
    return self.gen(noise)

print('Generator class created!')

# Noise Generator
def genNoise(number, z_dim):
  return torch.randn(number, z_dim).to(device) # Run standard normal distribution and store the result in the GPU

print('Noise generator function created!')

### Discriminator Model

---


In [ ]:
# Discriminator block
def discBlock(inp, out):
  return nn.Sequential(
      nn.Linear(inp, out),
      nn.LeakyReLU(0.2) # Prevent neurons from dying by allowing small negative values to pass
  )

print('discBlock function created!')

#Discriminator
class Discriminator(nn.Module):
  def __init__(self, i_dim=784, h_dim=256):
    super().__init__()
    self.disc = nn.Sequential(
        discBlock(i_dim, h_dim*4), # Input Size is 784 and Exit Size is 1024
        discBlock(h_dim*4, h_dim*2), # Input Size is 1024 and Exit Size is 512
        discBlock(h_dim*2, h_dim), # Input Size is 512 and Exit Size is 256
        nn.Linear(h_dim, 1) # Input Size is 256 and Exit Size is 1
    )

  def forward(self, image):
    return self.disc(image)

print('Discriminator class created!')

### Optimizer Function

---


In [ ]:
# Params
gen = Generator(z_dim).to(device)
gen_opt = torch.optim.Adam(gen.parameters(), lr=lr)
disc = Discriminator().to(device)
disc_opt = torch.optim.Adam(disc.parameters(), lr=lr)

print('Optimizer params set!')

In [ ]:
# Show generator structure
gen

In [ ]:
# Show discriminator structure
disc

### Test Function

---


In [ ]:
x,y = next(iter(dataLoader)) # Get a batch of 128 images
# Show the shape of the data
print(x.shape, y.shape)
# Show first 10 labels
print(y[:10])

In [ ]:
# Show grid
noise = genNoise(bs, z_dim)
fake = gen(noise)
showGrid(fake)

### Calculate the Loss

---


In [ ]:
# Generator Loss
def calc_gen_loss(loss_func, gen, disc, number, z_dim):
  gen_noise = genNoise(number, z_dim)
  gen_fake = gen(gen_noise)
  gen_pred = disc(gen_fake)
  gen_targets = torch.ones_like(gen_pred) # Create a tensor with dimensionalities similar to predictions and fill them with 1's
  gen_loss = loss_func(gen_pred, gen_targets)

  return gen_loss

print('calc_gen_loss function created!')

# Discriminator Loss
def calc_disc_loss(loss_func, gen, disc, number, real, z_dim):
  noise = genNoise(number, z_dim)
  # Fake Images
  fake = gen(noise)
  disc_fake = disc(fake.detach()) # Don't change or tweak generator params when backpropogating
  disc_fake_targets = torch.zeros_like(disc_fake) # Create a tensor with dimensionalities similar to predictions and fill them with 0's
  disc_fake_loss = loss_func(disc_fake, disc_fake_targets)
  # Real Images
  disc_real = disc(real)
  disc_real_targets = torch.ones_like(disc_real) # Create a tensor with dimensionalities similar to predictions and fill them with 1's
  disc_real_loss = loss_func(disc_real, disc_real_targets)
  # Final Loss
  disc_loss = (disc_fake_loss + disc_real_loss)/2

  return disc_loss

print('calc_disc_loss function created!')

### Training The Discriminator Model

---


In [ ]:
# Main loop
for epoch in range(epochs):
  for real, _ in tqdm(dataLoader):
    # Discriminator
    disc_opt.zero_grad() # Set gradient to `0`
    cur_bs = len(real) # Real: 128*1*28*28
    real = real.view(cur_bs, -1) # 128*784
    real = real.to(device)
    disc_loss = calc_disc_loss(loss_func, gen, disc, cur_bs, real, z_dim)
    disc_loss.backward(retain_graph=True) # Take the loss value and backpropogate it to calculate the gradients across the NN
    disc_opt.step() # Tweak and update discriminator params

    # Generator
    gen_opt.zero_grad() # Set gradient to `0`
    gen_loss = calc_gen_loss(loss_func, gen, disc, cur_bs, z_dim)
    gen_loss.backward(retain_graph=True) # Take the loss value and backpropogate it to calculate the gradients across the NN
    gen_opt.step() # Tweak and update generator params

    # Stats Visualization
    mean_disc_loss += disc_loss.item()/info_step
    mean_gen_loss += gen_loss.item()/info_step

    if cur_step % info_step == 0 and cur_step > 0:
      fake_noise = genNoise(cur_bs, z_dim)
      fake = gen(fake_noise)
      showGrid(fake)
      showGrid(real)
      print(f"{epoch}: step {cur_step} / Gen loss: {mean_gen_loss} / disc_loss: {mean_disc_loss}")
      mean_gen_loss, mean_disc_loss = 0,0
    
    cur_step += 1